# Load Data

In [2]:
from tensorflow import keras

(train_data, train_targets), (test_data, test_targets) = keras.datasets.boston_housing.load_data()


# Prepare Data

In [3]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

# Build Model


In [13]:
def build_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(64, activation = 'relu', input_shape=(train_data.shape[1],)))
    model.add(keras.layers.Dense(64, activation= 'relu'))
    model.add(keras.layers.Dense(1))
    model.compile(optimizer='rmsprop', loss = 'mse', metrics=['mae'])
    return model

# K Fold Validation



In [ ]:
import numpy as np

k = 4
num_val_samples = len(train_data)//k
num_epochs = 500
all_mae_histories = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[: i * num_val_samples],
        train_data[(i+1) * num_val_samples:]],
        axis = 0)
    partial_train_targets = np.concatenate(
    [train_targets[:i * num_val_samples],
    train_targets[(i + 1) * num_val_samples:]], axis = 0)
    
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets, epochs=num_epochs,
             batch_size = 1, verbose = 0)
    val_mse, val_mae = model.evaluate(val_data, val_targets,  verbose = 0)
    print (history.history.keys())
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

processing fold # 0


In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories] for i in range(num_epochs))
]